In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('seaborn-v0_8')

import torch
import scipy
import crossprob
from tqdm import tqdm
import random
import pickle as pkl

from prompt_risk.bounds import *
from prompt_risk.utils import *

In [2]:
from detoxify import Detoxify
toxicity_model = Detoxify(device="cuda:0")

/opt/conda/envs/prompt_risk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for chat_root in [
    "full", 
    "red_team"
]:
    print("Chat data:", chat_root)

    filepath = "../llm_output/{}_chat/google-flan-t5-xxl_predictions.csv".format(chat_root)
    
    chat_df = pd.read_csv(filepath)
    responses = chat_df["generated_text"].tolist()
    tox_scores = []
    bs = 32
    for idx in tqdm(range(len(responses)//bs+1)):
        r = [str(res) for res in responses[idx*bs:(idx+1)*bs]]
        if len(r) == 0:
            break
        try:
            tox_scores.extend(max(toxicity_model.predict(r).values()))
        except Exception as e:
            print(e)
            print(r)
            e += 7
            
    assert len(tox_scores) == len(responses)
    chat_df["toxicity"] = tox_scores
    chat_df.to_csv(filepath)

Chat data: full


/tmp/ipykernel_9577/1220817581.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  chat_df = pd.read_csv(filepath)
100%|█████████████████████████████████████████████████████████████████▉| 8875/8876 [17:07<00:00,  8.63it/s]


Chat data: red_team


/tmp/ipykernel_9577/1220817581.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  chat_df = pd.read_csv(filepath)
100%|██████████████████████████████████████████████████████████████████| 8811/8811 [08:26<00:00, 17.40it/s]
